In [1]:
import numpy as np
import cv2

In [5]:
# Define the dictionary to use
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_50)

# Set up video capture from default camera
cap = cv2.VideoCapture(1)

roi_corners = {}

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()

    # Detect markers in the frame
    corners, ids, rejected = cv2.aruco.detectMarkers(frame, dictionary)

    # Draw detected markers and their IDs on the frame
    if ids is not None:
        # cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        for i, id in enumerate(ids):
            if id == 1:
                roi_corners["A"] = [int(corners[i][0][2][0]), int(corners[i][0][2][1])] # Bottom right corner of ID 1
            if id == 2:
                roi_corners["D"] = [int(corners[i][0][3][0]), int(corners[i][0][3][1])] # Bottom left corner of ID 2
            if id == 3:
                roi_corners["C"] = [int(corners[i][0][0][0]), int(corners[i][0][0][1])] # Top left corner of ID 3
            if id == 4:
                roi_corners["B"] = [int(corners[i][0][1][0]), int(corners[i][0][1][1])] # Top right corner of ID 4
        
        if 1 in ids and 2 in ids and 3 in ids and 4 in ids:

            # Here, I have used L2 norm. You can use L1 also.
            width_AD = np.sqrt(((roi_corners["A"][0] - roi_corners["D"][0]) ** 2) + ((roi_corners["A"][1] - roi_corners["D"][1]) ** 2))
            width_BC = np.sqrt(((roi_corners["B"][0] - roi_corners["C"][0]) ** 2) + ((roi_corners["B"][1] - roi_corners["C"][1]) ** 2))
            # maxWidth = max(int(width_AD), int(width_BC))
            maxWidth = 640
            
            height_AB = np.sqrt(((roi_corners["A"][0] - roi_corners["B"][0]) ** 2) + ((roi_corners["A"][1] - roi_corners["B"][1]) ** 2))
            height_CD = np.sqrt(((roi_corners["C"][0] - roi_corners["D"][0]) ** 2) + ((roi_corners["C"][1] - roi_corners["D"][1]) ** 2))
            # maxHeight = max(int(height_AB), int(height_CD))
            maxHeight = 440
            
            input_pts = np.float32([roi_corners["A"], roi_corners["B"], roi_corners["C"], roi_corners["D"]])
            output_pts = np.float32([[0, 0],
                                    [0, maxHeight - 1],
                                    [maxWidth - 1, maxHeight - 1],
                                    [maxWidth - 1, 0]])
            
            # Compute the perspective transform M
            M = cv2.getPerspectiveTransform(input_pts,output_pts)
            frame = cv2.warpPerspective(frame,M,(maxWidth, maxHeight),flags=cv2.INTER_LINEAR)
        
        else:
           cv2.aruco.drawDetectedMarkers(frame, corners, ids)


    # Display the resulting frame
    cv2.imshow('frame', frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()